In [1]:
!pip install --upgrade bs4
!pip install beautifulsoup4==4.9.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 4.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
## 這兩個模組常一起用
import urllib.request            # request web for info
from bs4 import BeautifulSoup    # provides a parse tree

In [3]:
from google.colab import drive
# directory
drive.mount('/content/gdrive')

# save as .json
import json

Mounted at /content/gdrive


In [4]:
page_start = 1679
page_end = 9024
n_page = page_end - page_start + 1
n_sticky = 2

# 2022/10/03 page.1679~9024 includede 7m~4yr ago
# 2022/10/02 page/9021 包含到 4yr ago 跟 5yr ago

In [ ]:
# 第一步驟
# 一律從首頁進入，並爬取主題連結

def EJMRpages(page_start = page_start, page_end = page_end):
  """用來爬文章主題的連結

  Parameters
  ----------
  n_pages = 1 (num)
      要爬幾頁文章

  Returns
  -------
  href: (list)
      文章主題連結, a list of url

  Notes
  -----
  一頁40個主題，主題若包含多頁也會一起存起來
  """
  topic_main_element = []

  if page_start == 1:
      
      topic_main = "https://www.econjobrumors.com" # main page
      
      req = urllib.request.Request(topic_main)
      req_open = urllib.request.urlopen(req)
      soup = BeautifulSoup(req_open, "html.parser")
      topic_main_element.extend(list(
          soup.select('#front-page #main #discussions table tr td:first-child a[href^="https"]:first-child')
          ))
          
      # 包含2篇置頂文章，len=42
      for p in range(2, n_page+1): # 使用for迴圈翻頁 
          topic_n = "https://www.econjobrumors.com/page/"+ str(p)
          
          req = urllib.request.Request(topic_n)
          req_open = urllib.request.urlopen(req)
          soup = BeautifulSoup(req_open, "html.parser")
          topic_main_element.extend( list(
              soup.select('#front-page #main #discussions table tr td:first-child a[href^="https"]:first-child')
                                            ))
      # transfrom type of topic_pages from a list of set to a list of str 
      topic_pages = list( topic_main_element[i]['href'] for i in range(len(topic_main_element) ))

  else:
      # 包含2篇置頂文章，len=42
      for p in range(page_start, page_end+1): # 使用for迴圈翻頁 
          topic_n = "https://www.econjobrumors.com/page/"+ str(p)
          
          req = urllib.request.Request(topic_n)
          req_open = urllib.request.urlopen(req)
          soup = BeautifulSoup(req_open, "html.parser")
          topic_main_element.extend( list(
              soup.select('#front-page #main #discussions table tr td:first-child a[href^="https"]:first-child')
                                            ))
      # transfrom type of topic_pages from a list of set to a list of str 
      topic_pages = list( topic_main_element[i]['href'] for i in range(len(topic_main_element) ))

  return topic_pages
    
topicURL = EJMRpages(page_start = page_start, page_end = page_end)
print(len(topicURL))  #293840


# save the file to Google Drive
with open("/content/gdrive/MyDrive/論文相關材料/EJMRurl.json", "w") as outfile:
    json.dump(topicURL, outfile)

293840


In [5]:
import json
with open("/content/gdrive/MyDrive/論文相關材料/EJMRurl.json") as jsonfile:
    topicURL = json.load(jsonfile)

In [ ]:
topicURL[15000]

'https://www.econjobrumors.com/topic/government-pays-me-to-attend-aboriginal-culture-workshops-for-a-week'

In [6]:
# -*- coding: utf-8 -*-
    
# 第二步驟
def EJMRtopics(topicURL, n_sticky):
  """用來爬文章主題，文章內容

  Parameters
  ----------
  n_sticky = 2
      有幾篇置頂文章
      sum(pages of posts)

  Returns
  -------
  Result, dict('topic', 'posts')
  文章主題，文章內容

  Notes
  -----
  須注意置頂文章可能有多頁
  """

  # delete sticky notes
  # print(topicURL[:n_sticky])
  # del topicURL[:n_sticky]
  print(f' There are {len(topicURL)} topics remaining after remove StickyNotes.')


  topics = list()
  posts = list()

  for topic in topicURL:

      topics.append(topic)

      # here we are at the first page
      req = urllib.request.Request(topic)
      page = urllib.request.urlopen(req)
      soup = BeautifulSoup(page, "html.parser")

      div_post = soup.select('.post p')
      for dp in div_post:
          if dp.parent.name == 'blockquote':
              dp.extract()

      raw_post = list( div_post[i].text.strip() for i in range(len(div_post)) ) 

      posts.append(raw_post)

      # print('1')

      # if there's "nav", that is, there are multiple pages in the topic
      # this for loop runs under topics.
      # delete-able? because nav contain net page?
      if soup.find("div", class_ = "nav"):
          # this for loop runs under pages in topic. 
          # if there is still 'next page' icon, shift to next page
          
          p = 2
          # print('2')
          while soup.find_all("a", class_="next page-numbers"):
      
              topic_n = topic + "/page/" + str(p)
              
              req = urllib.request.Request(topic_n)

              try:
                page = urllib.request.urlopen(req)
              except:
                pass
              
              soup = BeautifulSoup(page, "html.parser")
          
              div_post = soup.select('.post p')
              for dp in div_post:
                  if dp.parent.name == 'blockquote':
                      dp.extract()

              raw_post = list( div_post[i].text.strip() for i in range(len(div_post)) )  
          
          
              posts[-1].extend(raw_post)
          
              p += 1
          
              print( f'{p}')
          
          print(f' There are {p-1} pages in {topic}. ')
      
      else:  # only one page
          print(f' There is only one page in {topic}. ')
      
  result = {'topic' : topics,
        'posts' : posts}

  return result
# dataset = EJMRdata(page_start = 1679, page_end =9024 ,n_sticky = 2)


    

In [12]:
EJMRpost = EJMRtopics(topicURL=topicURL[40000:50000], n_sticky=n_sticky)

Streaming output truncated to the last 5000 lines.
3
 There are 2 pages in https://www.econjobrumors.com/topic/add-more-variables-variable-of-interest-becomes-statistically-significant. 
 There is only one page in https://www.econjobrumors.com/topic/circle-back-with-jen-psaki-a-new-day-time-talk-show. 
 There is only one page in https://www.econjobrumors.com/topic/predictions-of-bitcoin-crash. 
 There is only one page in https://www.econjobrumors.com/topic/is-kasy-and-sautmann-econometrica-2021-really-an-econometrics-paper. 
 There is only one page in https://www.econjobrumors.com/topic/us-economy-teeters-on-the-edge-of-recession. 
 There is only one page in https://www.econjobrumors.com/topic/hooters-under-fire-over-new-%e2%80%98crotch-string%e2%80%99-uniform-shorts. 
 There is only one page in https://www.econjobrumors.com/topic/bacon-2. 
 There is only one page in https://www.econjobrumors.com/topic/netflix-fires-black-trans-and-pregnant-organizer-of-employee-walkout. 
 There is onl

 There is only one page in https://www.econjobrumors.com/topic/convex-analysis-1. 
 There is only one page in https://www.econjobrumors.com/topic/my-bros-just-want-to-start-a-family-and-be-happy. 
 There is only one page in https://www.econjobrumors.com/topic/im-going-on-a-date-with-a-french-girl. 
 There is only one page in https://www.econjobrumors.com/topic/los-angeles-allows-mask-removal-if-alone-in-a-room-or-showeringswimming. 
 There is only one page in https://www.econjobrumors.com/topic/potato-sandwich-on-potato-bread-with-potato-salad-and-potato-beer. 
 There is only one page in https://www.econjobrumors.com/topic/are-malaysian-hans-representative-of-hans. 
3
4
5
6
 There are 5 pages in https://www.econjobrumors.com/topic/karlstack-goes-mainstream-gets-mentioned-by-name-on-tucker. 
 There is only one page in https://www.econjobrumors.com/topic/the-news-is-still-obsessed-with-d-j-t. 
 There is only one page in https://www.econjobrumors.com/topic/gov-kristi-noem-had-an-affair-wi

In [13]:
EJMRpost['posts'][0]

['Breaking News:',
 'The main researcher of the so-called Steele dossier was arrested as part of the special counsel inquiry led by John Durham.',
 'Step 1. No one says Russian collusion is happening',
 "If it's a hoax, why are you pulling your hair out over it?",
 "If it's a hoax, why are you pulling your hair out over it?",
 'Because the hoaxers must be held to account to save democracy',
 "Russia hating Ukrainian concocted yet another fake news about Russia, what's new?",
 'Because the hoaxers must be held to account to save democracy',
 'Correct!',
 'Because the hoaxers must be held to account to save democracy',
 'Bro, your guy called up election officials and pressured them to "find votes".',
 'Because the hoaxers must be held to account to save democracy',
 'Bro, your guy called up election officials and pressured them to "find votes".',
 'Then what happened?']

In [15]:
# save the file to Google Drive
with open("/content/gdrive/MyDrive/論文相關材料/EJMRpost7.json", "w") as outfile:
    json.dump(EJMRpost, outfile)

In [18]:
print(EJMRpost['topic'][0],EJMRpost['posts'][0])

https://www.econjobrumors.com/topic/nytimes-key-leader-of-the-russian-hoax-is-arrested ['Breaking News:', 'The main researcher of the so-called Steele dossier was arrested as part of the special counsel inquiry led by John Durham.', 'Step 1. No one says Russian collusion is happening', "If it's a hoax, why are you pulling your hair out over it?", "If it's a hoax, why are you pulling your hair out over it?", 'Because the hoaxers must be held to account to save democracy', "Russia hating Ukrainian concocted yet another fake news about Russia, what's new?", 'Because the hoaxers must be held to account to save democracy', 'Correct!', 'Because the hoaxers must be held to account to save democracy', 'Bro, your guy called up election officials and pressured them to "find votes".', 'Because the hoaxers must be held to account to save democracy', 'Bro, your guy called up election officials and pressured them to "find votes".', 'Then what happened?']


In [16]:
# try to read the json file 
with open("/content/gdrive/MyDrive/論文相關材料/EJMRpost7.json") as jsonfile:
    test = json.load(jsonfile)

In [17]:
test['topic'][0]

'https://www.econjobrumors.com/topic/nytimes-key-leader-of-the-russian-hoax-is-arrested'